<a href="https://colab.research.google.com/github/Walt9819/factores-impacto-desempleo-mexico/blob/main/Python/FactoresDeImpactoEnDesempleoMX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equipo 18

BEDU - Santander: Módulo 2. Procesamiento de Datos con Python

Proyecto del equipo 18

**Integrantes:**
- Irene López [ver perfil](https://www.linkedin.com/in/irene-lopez-rodriguez/)
- Laura Lozano [ver perfil](https://www.linkedin.com/in/laura-lozano-bautista/)
- Enrique Rodríguez [ver perfil](https://www.linkedin.com/in/enrique-rodriguez97/)
- Walter Rosales [ver perfil]()

# Factores de Impacto en el Desempleo y la Recuperación Económica en México

## Objetivo
Analizar la situación del empleo en México durante los tres primeros trimestres de 2020, asimismo, calcular la probabilidad de estar desempleado conforme a características sociodemográficas y así identificar los perfiles de los mexicanos más vulnerables ante el deterioro del mercado laboral derivado de la situación que vivimos hoy en día.

# Primera descarga de datos desde el INEGI

**NOTA:** solo descargar si no están los datos dispoibles en el directorio de trabajo

In [ ]:
# run just once
!pip3 install zipfile38
!pip3 install dbfread

In [84]:
import requests, sys, io # lectura desde URL
import zipfile38 as zipfile # apoyos para lecturas de ZIP

import pandas as pd # usar dataframes
import os # manipulación de archivos en el sistema

from dbfread import DBF # lectura de archivos .dbf

zips = ["SDEMT0420", "SDEMT0520", "SDEMT0620", "SDEMT120", "enoen_sdemt320", "SDEMT420"]

for zip in zips:
  r = requests.get(f"https://github.com/Walt9819/factores-impacto-desempleo-mexico/blob/main/R/Project/enoe_sdem/{zip}.zip?raw=true")
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall("")


# Columnas a leer
columnas = ["ENT", "MUN", "SEX", "EDA", "NIV_INS", "RAMA", "CLASE2", "PER"]

# Conjunto de datos a llenar
data = pd.DataFrame()

# Leer todos los archivos `.dbf`
for file in os.listdir():
    if file.endswith(".dbf"):
      table = DBF(file, load=True)
      print(f"Load {file} succesfully\nTransforming data..")
      table = pd.DataFrame(iter(table))
      print("Adding data to main dataframe...")
      data = data.append(table[columnas])

#Export to data
data.to_csv("data_enoe.csv", encoding="utf-8")

Load SDEMT0420.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load SDEMT120.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load enoen_sdemt320.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load SDEMT0620.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load SDEMT0520.dbf succesfully
Transforming data..
Adding data to main dataframe...
Load ENOEN_SDEMT420.dbf succesfully
Transforming data..
Adding data to main dataframe...
  ENT  MUN SEX EDA  NIV_INS  RAMA  CLASE2  PER
0  09  007   1  48        4     4       1  220
1  09  007   2  37        4     0       4  220
2  09  007   1  17        3     0       4  220
3  09  007   1  16        3     0       2  220
4  09  007   2  87        3     0       4  220


# Lectura de datos

In [88]:
datos = pd.read_csv("data_enoe.csv", encoding="utf-8")

## **TODO**

```r
for (zfile in list.files(pattern = "*.zip$", recursive = TRUE)) {
  unzip(zfile, exdir = outDir)
}
```

###### Lectura de todos los archivos .dbf en la carpeta del proyecto
```r
rawdata <- lapply(list.files(pattern = "*.dbf$", recursive = TRUE), read.dbf)
```

###### Extraccion de atributos considerados para el modelo de regresion logistica y lineal. 
```r
selecteddata <- lapply(rawdata, select, c("ENT", "MUN", "SEX", "EDA", "NIV_INS", "RAMA", "CLASE2", "PER"))
```

###### Construccion de data frame y cambio de nombres
```r
data_enoe <- do.call(rbind, selecteddata)
colnames(data_enoe) <- c("cve_ent", "cve_mun", "sex", "eda", "niv_ins", "rama", "clase2", "per")
```

###### Se omiten valores NaN dentro de la base de datos.
```r
data_enoe <- na.omit(data_enoe)
```

###### Se omiten los registros de las personas no economicamente activas y no dispibles [clase2 == 4], a los menores de 15 años y mayores de 65 años [eda <= 15 & eda >= 65] y aquellos sin nivel educativo registrado [niv_ins == 5]
```r
data_enoe <- data_enoe[data_enoe$clase2 <= 3 & 
                         data_enoe$clase2 != 0 & 
                         data_enoe$eda >= 15 & 
                         data_enoe$eda <= 65 &
                         data_enoe$niv_ins <= 4, ]
```